# Preprocess on all data

- Drop null values
- Preprocess texts (tokenization, lemmatization, stop-word removal).

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Define the paths
f = "/content/drive/MyDrive/the-reddit-climate-change-dataset-comments.csv" #original data
output = "/content/drive/MyDrive/all_data.csv" #output file

In [7]:
# Packages imported
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [8]:
df = pd.read_csv(f)
c = df[['id','body','sentiment']]

In [ ]:
df.dropna(subset=['sentiment'],
axis=0, 
how='any', 
inplace=True 
)

In [9]:
# Text processing (tokenization, stop words removal)
def preProcess(doc):
    """
    Parameters
    ----------
    doc : List of String
        A document to process.
    Returns the processed document : [0]: a list of words from the basic sentences; [1]: list of tokenized words; [2]: a string of of tokenized words.
    -------
    TYPE
        Process and return a document (tokenization, lemmatization, stop-word removal).
    """
        
    Stopwords = list(set(nltk.corpus.stopwords.words('english')))
    stemmer = SnowballStemmer("english")
    WN_lemmatizer = WordNetLemmatizer()

    sentences = sent_tokenize(doc)
    tokens = []
    for sentence in sentences:
        words = word_tokenize(sentence)
        words = [stemmer.stem(word) for word in words]
        words = [WN_lemmatizer.lemmatize(word, pos="v") for word in words]

        words = [word for word in words if word.isalpha() and word not in Stopwords]  # get rid of numbers and Stopwords
        tokens.extend(words)

    return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [13]:
c['c_body'] = c.apply(lambda row : preProcess(row['body']), axis = 1)

<ipython-input-13-6d3de48c9535>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c['c_body'] = c.apply(lambda row : preProcess(row['body']), axis = 1)


In [15]:
c.dropna(subset=['sentiment'],
axis=0, 
how='any', 
inplace=True 
)

<ipython-input-15-73bf6a60bbed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c.dropna(subset=['sentiment','c_body'],


In [28]:
df['body_cleaned'] = c['c_body']

In [30]:
out = df.drop(['body'],axis=1)

In [32]:
out.columns

Index(['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw',
       'created_utc', 'permalink', 'sentiment', 'score', 'body_cleaned'],
      dtype='object')

In [34]:
out.to_csv(output,index=False,encoding='utf-8-sig')